# Lab 5: Leveraging Open Data from Wikipedia for LLM Prompt Engineering

## Overview
This lab demonstrates how to extract structured data from Wikipedia pages and use it to create effective prompts for Large Language Models (LLMs). You'll learn to work with real-world financial data, process it programmatically, and engineer prompts for various AI tasks.

## Learning Objectives
- ✓ Extract financial index components from Wikipedia
- ✓ Retrieve company infobox data programmatically
- ✓ Build structured datasets from semi-structured web data
- ✓ Design effective LLM prompts for different tasks
- ✓ Process and clean text data for AI consumption
- ✓ Create reusable prompt templates and utilities

## Part 1: Data Extraction from Wikipedia

### What is a Financial Index?
A financial index is a composite measure of a subset of companies in a specific market or sector. Examples include:
- **S&P 500**: 500 largest US companies
- **EURO STOXX 50**: 50 largest Eurozone companies
- **DAX**: 40 largest German companies

### Your Task
1. **Identify components**: Extract the list of companies in each index from Wikipedia
2. **Gather company data**: Retrieve detailed information (infoboxes) from each company's Wikipedia page
3. **Build a dataset**: Combine all data into structured format suitable for LLM processing
4. **Engineer prompts**: Create effective prompts that leverage this data for AI tasks

### Data Sources
- **Index components**: Wikipedia articles listing index members
- **Company data**: Wikipedia infoboxes (structured data boxes on company pages)
- **Dump file**: Optional - for advanced analysis of full Wikipedia articles

### Optional: Full Wikipedia Dump
For advanced analysis, you can download the complete Wikipedia dump from:
- **Link**: https://dumps.wikimedia.org/enwiki/
- **File**: `enwiki-latest-pages-articles-multistream-index.txt.bz2`
- **Use case**: Full-text search, article history analysis, or complete data scraping
- **Note**: Very large files (100+ GB) - requires significant storage and processing power

For this lab, we'll focus on extracting specific data via the Wikipedia API, which is more efficient.

In [15]:
# ============================================================================
# IMPORTS & SETUP
# ============================================================================
# These libraries enable us to work with Wikipedia data

import pandas as pd              # Data manipulation and analysis
import urllib.request           # HTTP requests to Wikipedia
from pathlib import Path        # Cross-platform file path handling
from typing import Union        # Type hints for better code clarity
from tqdm import tqdm          # Progress bars for long operations
import wptools               # Wikipedia parsing (infobox extraction)
from loguru import logger       # Enhanced logging
import json                     # Working with JSON data
import numpy as np             # Numerical operations

import os

## Step 1: Extract Index Components from Wikipedia

### Task: Extract Company Lists
We'll extract the list of companies that make up each financial index directly from Wikipedia.

### Indices We're Covering:
1. **S&P 500** (USA) - 500 largest US companies
2. **EURO STOXX 50** (Europe) - 50 largest Eurozone companies  
3. **CAC 40** (France) - 40 largest French companies
4. **DAX** (Germany) - 40 largest German companies
5. **CSI 300** (China) - 300 largest Chinese companies
6. **S&P Latin America 40** (Latin America) - 40 major LA companies
7. **BSE SENSEX** (India) - 30 largest Indian companies
8. **NASDAQ-100** (USA Tech) - 100 largest non-financial NASDAQ companies

### How It Works:
- Each index has a Wikipedia article with a table listing its components
- We'll use `pd.read_html()` to extract all tables from these pages
- Tables are saved as CSV files for later processing
- This approach is fast, requires no authentication, and respects Wikipedia's terms

In [16]:
# ============================================================================
# FUNCTION 1: Extract Tables from Wikipedia
# ============================================================================
# This function downloads tables from Wikipedia articles and saves them locally

def get_index_components(wiki_url: str, save_dir: Union[str, Path], 
                         opener: urllib.request.OpenerDirector) -> None:
    """
    Extract all HTML tables from a Wikipedia page and save as CSV files.
    
    Parameters:
    -----------
    wiki_url : str
        The Wikipedia page URL to scrape (e.g., list of index components)
    save_dir : Union[str, Path]
        Directory where CSV files will be saved
    opener : urllib.request.OpenerDirector
        Custom URL opener with proper User-Agent headers
        
    Output:
    -------
    Creates CSV files named table_0.csv, table_1.csv, etc. in save_dir
    Each file contains one table from the Wikipedia page
    
    Example:
    --------
    >>> get_index_components(
    ...     "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    ...     "./data/indices/sp500",
    ...     opener
    ... )
    """
    # Create the save directory if it doesn't exist
    save_path = Path(save_dir)
    save_path.mkdir(parents=True, exist_ok=True)
    
    try:
        # Read all HTML tables from the Wikipedia page
        tables = pd.read_html(wiki_url)
        
        # Save each table as a separate CSV file
        for idx, table in enumerate(tables):
            csv_path = save_path / f"table_{idx}.csv"
            table.to_csv(csv_path, index=False, encoding='utf-8')
            logger.info(f"Saved table {idx} to {csv_path}")
            
    except Exception as e:
        logger.error(f"Error extracting tables from {wiki_url}: {e}")
        raise

In [17]:
# ============================================================================
# SETUP: Configure Wikipedia Index URLs and HTTP Headers
# ============================================================================

# Dictionary mapping index names to their Wikipedia article URLs
# These URLs contain tables with the company components of each index
indices = {
    "sp500": "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    "eurostoxx50": "https://en.wikipedia.org/wiki/EURO_STOXX_50",
    "cac40": "https://en.wikipedia.org/wiki/CAC_40",
    "dax": "https://en.wikipedia.org/wiki/DAX",
    "csi300": "https://en.wikipedia.org/wiki/CSI_300_Index",
    "spla40": "https://en.wikipedia.org/wiki/S%26P_Latin_America_40",
    "bsesensex": "https://en.wikipedia.org/wiki/BSE_SENSEX",
    "nasdaq100": "https://en.wikipedia.org/wiki/Nasdaq-100",
}

# IMPORTANT: Configure HTTP headers to identify our bot to Wikipedia
# This is REQUIRED for ethical web scraping - identify yourself!
# Wikipedia may block requests without proper User-Agent headers

opener = urllib.request.build_opener()
opener.addheaders = [
    ("User-Agent", "MyResearchBot/1.0 (contact@example.com)")  # Identify your bot
]
urllib.request.install_opener(opener)

In [18]:
# ============================================================================
# EXECUTION: Download Index Components
# ============================================================================
# Loop through each index and extract its company components from Wikipedia
# This may take a few minutes depending on internet speed

for idx, url in indices.items():
    print(f"Processing index: {idx}...")
    try:
        get_index_components(url, f"./data/indices/{idx}", opener)
        print(f"  -> Done: Components for {idx} saved.")
    except Exception as e:
        print(f"  !! Error processing {idx}: {e}")


Processing index: sp500...


2025-11-29 14:05:48.946 | INFO     | __main__:get_index_components:45 - Saved table 0 to data/indices/sp500/table_0.csv
2025-11-29 14:05:48.948 | INFO     | __main__:get_index_components:45 - Saved table 1 to data/indices/sp500/table_1.csv
2025-11-29 14:05:48.950 | INFO     | __main__:get_index_components:45 - Saved table 2 to data/indices/sp500/table_2.csv


  -> Done: Components for sp500 saved.
Processing index: eurostoxx50...


2025-11-29 14:05:49.184 | INFO     | __main__:get_index_components:45 - Saved table 0 to data/indices/eurostoxx50/table_0.csv
2025-11-29 14:05:49.185 | INFO     | __main__:get_index_components:45 - Saved table 1 to data/indices/eurostoxx50/table_1.csv
2025-11-29 14:05:49.186 | INFO     | __main__:get_index_components:45 - Saved table 2 to data/indices/eurostoxx50/table_2.csv
2025-11-29 14:05:49.187 | INFO     | __main__:get_index_components:45 - Saved table 3 to data/indices/eurostoxx50/table_3.csv
2025-11-29 14:05:49.188 | INFO     | __main__:get_index_components:45 - Saved table 4 to data/indices/eurostoxx50/table_4.csv
2025-11-29 14:05:49.189 | INFO     | __main__:get_index_components:45 - Saved table 5 to data/indices/eurostoxx50/table_5.csv
2025-11-29 14:05:49.189 | INFO     | __main__:get_index_components:45 - Saved table 6 to data/indices/eurostoxx50/table_6.csv
2025-11-29 14:05:49.190 | INFO     | __main__:get_index_components:45 - Saved table 7 to data/indices/eurostoxx50/tabl

  -> Done: Components for eurostoxx50 saved.
Processing index: cac40...
  -> Done: Components for cac40 saved.
Processing index: dax...


...

  -> Done: Components for spla40 saved.
Processing index: bsesensex...


2025-11-29 14:05:50.023 | INFO     | __main__:get_index_components:45 - Saved table 0 to data/indices/bsesensex/table_0.csv
2025-11-29 14:05:50.025 | INFO     | __main__:get_index_components:45 - Saved table 1 to data/indices/bsesensex/table_1.csv
2025-11-29 14:05:50.026 | INFO     | __main__:get_index_components:45 - Saved table 2 to data/indices/bsesensex/table_2.csv
2025-11-29 14:05:50.026 | INFO     | __main__:get_index_components:45 - Saved table 3 to data/indices/bsesensex/table_3.csv
2025-11-29 14:05:50.027 | INFO     | __main__:get_index_components:45 - Saved table 4 to data/indices/bsesensex/table_4.csv
2025-11-29 14:05:50.028 | INFO     | __main__:get_index_components:45 - Saved table 5 to data/indices/bsesensex/table_5.csv
2025-11-29 14:05:50.028 | INFO     | __main__:get_index_components:45 - Saved table 6 to data/indices/bsesensex/table_6.csv
2025-11-29 14:05:50.029 | INFO     | __main__:get_index_components:45 - Saved table 7 to data/indices/bsesensex/table_7.csv
2025-11-

  -> Done: Components for bsesensex saved.
Processing index: nasdaq100...


2025-11-29 14:05:50.401 | INFO     | __main__:get_index_components:45 - Saved table 0 to data/indices/nasdaq100/table_0.csv
2025-11-29 14:05:50.402 | INFO     | __main__:get_index_components:45 - Saved table 1 to data/indices/nasdaq100/table_1.csv
2025-11-29 14:05:50.403 | INFO     | __main__:get_index_components:45 - Saved table 2 to data/indices/nasdaq100/table_2.csv
2025-11-29 14:05:50.403 | INFO     | __main__:get_index_components:45 - Saved table 3 to data/indices/nasdaq100/table_3.csv
2025-11-29 14:05:50.404 | INFO     | __main__:get_index_components:45 - Saved table 4 to data/indices/nasdaq100/table_4.csv
2025-11-29 14:05:50.406 | INFO     | __main__:get_index_components:45 - Saved table 5 to data/indices/nasdaq100/table_5.csv
2025-11-29 14:05:50.406 | INFO     | __main__:get_index_components:45 - Saved table 6 to data/indices/nasdaq100/table_6.csv
2025-11-29 14:05:50.407 | INFO     | __main__:get_index_components:45 - Saved table 7 to data/indices/nasdaq100/table_7.csv
2025-11-

  -> Done: Components for nasdaq100 saved.


## Step 2: Extract Company Infoboxes from Wikipedia

### What are Infoboxes?
Wikipedia infoboxes are structured data boxes that appear on the right side of articles. They contain:
- Company name and alternative names
- Industry classification
- Founded date and location
- Key executives
- Headquarters location
- Number of employees
- Revenue and financial metrics
- Official website URLs
- Stock exchange listings
- And much more...

### Why Infoboxes?
- **Structured data**: Unlike article body text, infoboxes are semi-structured
- **Consistency**: Fields follow a template across similar articles
- **Ease of extraction**: Wikipedia APIs can parse infoboxes directly
- **Rich context**: Perfect for LLM prompts - contains exactly the info LLMs need

### Process
1. Use the `wptools` library to fetch each company's Wikipedia page
2. Extract the infobox (structured data) from the page parse
3. Save as JSON for flexibility and later processing
4. Handle errors gracefully (some companies may not have Wikipedia pages)

In [19]:
# ============================================================================
# EXAMPLE: Extract a Single Company Infobox
# ============================================================================
# This example shows the process for one company (3M) from S&P 500
# In production, we'd loop this for all companies


# We'll use 3M (company name) as our test case
company_name = "3M"

def extract_save_infobox(company_name, output_folder="./data/infoboxes/sp500"):
    """
    Fetches Wikipedia infobox data for a company and saves as JSON in specified folder.
    """
    wiki_page = wptools.page(company_name, silent=True)
    wiki_page.get_parse(show=False)
    infobox = wiki_page.data.get('infobox', {})
    os.makedirs(output_folder, exist_ok=True)
    output_path = f"{output_folder}/{company_name.replace('/', '_')}_infobox.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(infobox, f, ensure_ascii=False, indent=2)
    return output_path, infobox

# Example usage for 3M
output_path, infobox = extract_save_infobox(company_name)

print(f"Infobox for {company_name} saved to: {output_path}")



Infobox for 3M saved to: ./data/infoboxes/sp500/3M_infobox.json


In [20]:
# ============================================================================
# DISPLAY: View the Extracted Infobox
# ============================================================================
# This shows what data we extracted from Wikipedia

# Display the infobox data previously saved for 3M (or any company) as an example

print(f"Infobox for {company_name}:")
print(json.dumps(infobox, indent=2))

Infobox for 3M:
{
  "name": "3M Company",
  "logo": "3M wordmark.svg",
  "logo_size": "175px",
  "image": "3-M Building Maplewood MN1.jpg",
  "image_size": "250px",
  "image_caption": "3M headquarters in [[Maplewood, Minnesota]]",
  "former_name": "Minnesota Mining and Manufacturing Company (1902\u20132002)",
  "type": "[[Public company|Public]]",
  "traded_as": "{{Unbulleted list|New York Stock Exchange|MMM|[[Dow Jones Industrial Average|DJIA]] component|[[S&P 100]] component|[[S&P 500]] component}} {{New York Stock Exchange|MMM}}",
  "ISIN": "{{ISIN|sl|=|n|pl|=|y|US88579Y1010}}",
  "industry": "[[Conglomerate (company)|Conglomerate]]",
  "foundation": "{{Start date and age|1902|6|13}} in [[Two Harbors, Minnesota]], U.S.",
  "founders": "{{Unbulleted list|J. Danley Budd|Henry S. Bryan|William A. McGonagle|John Dwan|Hermon W. Cable | Charles Simmons|ref|{{cite web |url=https://www.3m.com.au/3M/en_AU/company-au/news-releases/full-story/?storyid=51f5cfac-3ea9-4a98-a406-e2b955c3fd40 |titl

## Step 3: Aggregate Infoboxes into Databases

### What We're Building
We're converting individual JSON files (one per company) into consolidated CSV databases (one per index).

### Why?
- **Easier analysis**: CSV format works with pandas, Excel, and most analysis tools
- **Efficiency**: One file per index instead of hundreds of individual JSON files
- **Standardization**: Creates a uniform dataset structure for LLM processing

### Process
1. Read all JSON infobox files for an index from disk
2. Convert each JSON to a DataFrame row
3. Concatenate all rows into a single DataFrame
4. Save as CSV with proper encoding

### Notes for Future Enhancement
- The infoboxes contain many fields beyond what we use now (URLs, images, etc.)
- Future work could extract and leverage additional information
- This foundation allows flexible data extraction later

In [21]:
# ============================================================================
# EXECUTION: Merge All Infoboxes into Index Databases
# ============================================================================
# Loop through each index folder and consolidate all JSON infoboxes into CSV

def aggregate_infoboxes(index_name: str, infobox_dir: str, output_dir: str = "./data/databases") -> None:
    """
    Aggregate all JSON infobox files for an index into a single CSV database.
    
    Parameters:
    -----------
    index_name : str
        Name of the index (e.g., 'sp500')
    infobox_dir : str
        Directory containing JSON infobox files
    output_dir : str
        Directory where consolidated CSV will be saved
    """
    infobox_path = Path(infobox_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    if not infobox_path.exists():
        logger.warning(f"Infobox directory {infobox_dir} does not exist. Skipping {index_name}.")
        return
    
    # Find all JSON files in the infobox directory
    json_files = list(infobox_path.glob("*_infobox.json"))
    
    if not json_files:
        logger.warning(f"No infobox JSON files found in {infobox_dir}")
        return
    
    rows = []
    skipped = 0
    
    for json_file in tqdm(json_files, desc=f"Processing {index_name}"):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                infobox_data = json.load(f)
            
            # Add the company name (from filename) and index name
            company_name = json_file.stem.replace('_infobox', '')
            row = {'company_name': company_name, 'index': index_name}
            
            # Flatten the infobox dictionary into the row
            for key, value in infobox_data.items():
                # Convert value to string if it's not already
                if isinstance(value, (dict, list)):
                    row[key] = json.dumps(value, ensure_ascii=False)
                else:
                    row[key] = str(value) if value is not None else ''
            
            rows.append(row)
            
        except Exception as e:
            logger.error(f"Error processing {json_file}: {e}")
            skipped += 1
            continue
    
    if rows:
        # Create DataFrame from all rows
        df = pd.DataFrame(rows)
        
        # Save to CSV
        csv_path = output_path / f"{index_name}_infoboxes.csv"
        df.to_csv(csv_path, index=False, encoding='utf-8')
        logger.info(f"{index_name}: saved {len(rows)} rows to {csv_path}. Skipped {skipped} files.")
        print(f">>> {index_name}: saved {len(rows)} rows to {csv_path}. Skipped {skipped} files.")
    else:
        logger.warning(f"No valid infoboxes found for {index_name}")

# Process S&P 500 infoboxes (example - you can loop through all indices)
aggregate_infoboxes("sp500", "./data/infoboxes/sp500")


Processing sp500: 100%|██████████| 1/1 [00:00<00:00, 652.30it/s]
2025-11-29 14:05:53.579 | INFO     | __main__:aggregate_infoboxes:68 - sp500: saved 1 rows to data/databases/sp500_infoboxes.csv. Skipped 0 files.


>>> sp500: saved 1 rows to data/databases/sp500_infoboxes.csv. Skipped 0 files.


# Part 2: Data Processing & LLM Prompt Engineering

## Overview of Part 2
Now that we have structured company data from Wikipedia, we'll:

1. **Load and analyze** the infobox database
2. **Clean and preprocess** the data for LLM consumption
3. **Create prompt templates** for different LLM tasks
4. **Design context formatting** that maximizes LLM effectiveness

## Key Concepts

### Why Clean Data for LLMs?
- LLMs perform better with well-structured, clean text
- Removing noise and formatting artifacts improves accuracy
- Consistent formatting allows for better prompt engineering
- Clean data enables batch processing and cost optimization

### Prompt Engineering
Prompt engineering is the art of crafting inputs to LLMs to get better outputs. We'll explore:
- **Context formatting**: How to present company data effectively
- **Task-specific templates**: Different prompts for different goals
- **Few-shot learning**: Providing examples to guide LLM behavior
- **Output structuring**: Getting structured responses (JSON, tables, etc.)


In [22]:
# ============================================================================
# STEP 4: LOAD AND ANALYZE INFOBOX DATA
# ============================================================================
# Now we'll load the consolidated CSV database and analyze its structure

def load_infobox_database(csv_path: str) -> pd.DataFrame:
    """
    Load infobox database from CSV file.
    
    Parameters:
    -----------
    csv_path : str
        Path to the CSV file containing infobox data
        
    Returns:
    --------
    pd.DataFrame
        DataFrame containing the infobox data
    """
    try:
        df = pd.read_csv(csv_path, encoding='utf-8')
        return df
    except FileNotFoundError:
        logger.error(f"File not found: {csv_path}")
        # Try alternative path
        alt_path = f"./data/databases/{Path(csv_path).name}"
        if Path(alt_path).exists():
            logger.info(f"Trying alternative path: {alt_path}")
            return pd.read_csv(alt_path, encoding='utf-8')
        raise

def analyze_infobox_data(df: pd.DataFrame) -> None:
    """
    Display basic statistics and information about the infobox dataset.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame containing infobox data
    """
    print(f"Loaded infobox dataset with {df.shape[0]:,} rows and {df.shape[1]:,} columns.")
    print("\n" + "="*80)
    print("DATASET OVERVIEW")
    print("="*80)
    print(f"\nColumns ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        non_null = df[col].notna().sum()
        print(f"  {i:2d}. {col:30s} ({non_null:,} non-null values)")
    
    print(f"\n\nFirst few rows:")
    print(df.head())
    
    print(f"\n\nData types:")
    print(df.dtypes)
    
    print(f"\n\nMissing values per column:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_pct
    })
    print(missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False))

# Load the S&P 500 infobox data (CSV format)
csv_path = "./data/databases/sp500_infoboxes.csv"
df = load_infobox_database(csv_path)

# Display basic statistics about the dataset
analyze_infobox_data(df)


Loaded infobox dataset with 1 rows and 26 columns.

DATASET OVERVIEW

Columns (26):
   1. company_name                   (1 non-null values)
   2. index                          (1 non-null values)
   3. name                           (1 non-null values)
   4. logo                           (1 non-null values)
   5. logo_size                      (1 non-null values)
   6. image                          (1 non-null values)
   7. image_size                     (1 non-null values)
   8. image_caption                  (1 non-null values)
   9. former_name                    (1 non-null values)
  10. type                           (1 non-null values)
  11. traded_as                      (1 non-null values)
  12. ISIN                           (1 non-null values)
  13. industry                       (1 non-null values)
  14. foundation                     (1 non-null values)
  15. founders                       (1 non-null values)
  16. location_city                  (1 non-null values)
  17

In [23]:
# ============================================================================
# STEP 5: DATA PREPROCESSING FOR LLM
# ============================================================================
# Functions to clean and format data for Large Language Models

from typing import Dict
import re

def clean_text(text: str) -> str:
    """
    Clean and normalize text for LLM input.
    
    Removes Wikipedia markup, extra whitespace, and formatting artifacts.
    This ensures clean, readable text for LLMs.
    
    Parameters:
    -----------
    text : str
        Raw text from Wikipedia infobox
        
    Returns:
    --------
    str
        Cleaned text suitable for LLM processing
    """
    if pd.isna(text) or text is None:
        return ""
    
    text = str(text)
    
    # Remove Wikipedia template syntax {{...}}
    text = re.sub(r'\{\{[^}]*\}\}', '', text)
    
    # Remove Wikipedia link syntax [[...|...]] or [[...]]
    text = re.sub(r'\[\[([^\]]*\|)?([^\]]+)\]\]', r'\2', text)
    
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove extra whitespace and newlines
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    # Remove common Wikipedia artifacts
    text = text.replace('{{', '').replace('}}', '')
    text = text.replace('|', ' ')
    
    return text


def extract_key_facts(row: pd.Series) -> Dict[str, str]:
    """
    Extract key facts from a company infobox row.
    
    Focuses on most important fields for LLM processing:
    - name, type, industry (what the company is)
    - founded, founder (history)
    - headquarters (location)
    - key_people (leadership)
    - employees, revenue (size/scale)
    - website, stock_exchange (external links)
    
    Uses case-insensitive matching to handle Wikipedia's varying field names.
    
    Parameters:
    -----------
    row : pd.Series
        A row from the infobox DataFrame
        
    Returns:
    --------
    Dict[str, str]
        Dictionary of key facts with cleaned values
    """
    facts = {}
    
    # Define key fields to extract (with variations)
    key_fields = {
        'name': ['name', 'company_name', 'title'],
        'type': ['type'],
        'industry': ['industry', 'sector', 'business'],
        'founded': ['founded', 'foundation', 'established', 'founded_date'],
        'founders': ['founders', 'founder'],
        'headquarters': ['headquarters', 'location_city', 'location_country', 'hq'],
        'key_people': ['key_people', 'ceo', 'leadership', 'executives'],
        'employees': ['num_employees', 'employees', 'employee_count'],
        'revenue': ['revenue', 'revenue_usd', 'annual_revenue'],
        'website': ['website', 'url', 'homepage'],
        'stock_exchange': ['traded_as', 'stock_exchange', 'exchange', 'ticker']
    }
    
    # Extract values using case-insensitive matching
    row_lower = {str(k).lower(): v for k, v in row.items()}
    
    for fact_name, field_variations in key_fields.items():
        for field_var in field_variations:
            # Try exact match first
            if field_var in row:
                value = row[field_var]
                if pd.notna(value) and str(value).strip():
                    facts[fact_name] = clean_text(str(value))
                    break
            # Try case-insensitive match
            elif field_var.lower() in row_lower:
                for orig_key, value in row.items():
                    if str(orig_key).lower() == field_var.lower():
                        if pd.notna(value) and str(value).strip():
                            facts[fact_name] = clean_text(str(value))
                            break
                if fact_name in facts:
                    break
    
    return facts


def row_to_context(row: pd.Series) -> str:
    """
    Convert a company row into a formatted context string for LLM input.
    
    Takes a DataFrame row and produces nicely formatted text that
    an LLM can understand and analyze. Format makes it clear what
    information is being provided.
    
    Output format:
    COMPANY INFORMATION:
    --
    FIELD: value
    FIELD: value
    
    Parameters:
    -----------
    row : pd.Series
        A row from the infobox DataFrame
        
    Returns:
    --------
    str
        Formatted context string for LLM
    """
    facts = extract_key_facts(row)
    
    if not facts:
        return "COMPANY INFORMATION:\n--\nNo information available."
    
    # Get company name (prefer 'name', fallback to 'company_name')
    company_name = facts.get('name', row.get('company_name', 'Unknown Company'))
    
    context_lines = [f"COMPANY INFORMATION: {company_name}", "--"]
    
    # Add each fact as a formatted line
    for key, value in facts.items():
        if value and value.strip():
            # Format field name nicely (capitalize, replace underscores)
            field_name = key.replace('_', ' ').title()
            context_lines.append(f"{field_name}: {value}")
    
    return "\n".join(context_lines)


# ============================================================================
# TEST: Demonstrate preprocessing on sample company
# ============================================================================

# Test on the first row of the dataset (or a specific company)
if not df.empty:
    sample_row = df.iloc[0]
    
    print("="*80)
    print("SAMPLE COMPANY DATA PREPROCESSING")
    print("="*80)
    
    print("\n1. Original Row (first few fields):")
    print(sample_row.head(10))
    
    print("\n2. Extracted Key Facts:")
    key_facts = extract_key_facts(sample_row)
    for key, value in key_facts.items():
        print(f"   {key:20s}: {value[:80]}..." if len(value) > 80 else f"   {key:20s}: {value}")
    
    print("\n3. Formatted Context for LLM:")
    context = row_to_context(sample_row)
    print(context)
else:
    print("No data available for testing. Please ensure infoboxes have been extracted.")


SAMPLE COMPANY DATA PREPROCESSING

1. Original Row (first few fields):
company_name                                                    3M
index                                                        sp500
name                                                    3M Company
logo                                               3M wordmark.svg
logo_size                                                    175px
image                               3-M Building Maplewood MN1.jpg
image_size                                                   250px
image_caption          3M headquarters in [[Maplewood, Minnesota]]
former_name      Minnesota Mining and Manufacturing Company (19...
type                                     [[Public company|Public]]
Name: 0, dtype: object

2. Extracted Key Facts:
   name                : 3M Company
   type                : Public
   industry            : Conglomerate
   founded             : in Two Harbors, Minnesota, U.S.
   founders            :  
   headquarters      

In [24]:
# ============================================================================
# STEP 6: CREATE PROMPT TEMPLATES FOR LLM TASKS
# ============================================================================
# Different tasks require different prompt structures. Build task-specific
# prompt templates that can be reused across your dataset.

def create_qa_prompt(company_context: str, question: str) -> str:
    """
    Create a question-answering prompt for LLM.
    
    Parameters:
    -----------
    company_context : str
        Formatted company information context
    question : str
        Question to ask about the company
        
    Returns:
    --------
    str
        Complete prompt for QA task
    """
    prompt = f"""You are a financial analyst assistant. Answer the following question based on the company information provided.

{company_context}

Question: {question}

Answer:"""
    return prompt


def create_classification_prompt(company_context: str, categories: list) -> str:
    """
    Create a classification prompt for LLM.
    
    Parameters:
    -----------
    company_context : str
        Formatted company information context
    categories : list
        List of categories to classify into
        
    Returns:
    --------
    str
        Complete prompt for classification task
    """
    categories_str = ", ".join(categories)
    prompt = f"""You are a business classification expert. Classify the following company into one of these categories: {categories_str}

{company_context}

Classification (choose one from: {categories_str}):"""
    return prompt


def create_summarization_prompt(company_context: str, focus: str = "general") -> str:
    """
    Create a summarization prompt for LLM.
    
    Parameters:
    -----------
    company_context : str
        Formatted company information context
    focus : str
        Focus area for summary (e.g., "financial", "operations", "general")
        
    Returns:
    --------
    str
        Complete prompt for summarization task
    """
    focus_instructions = {
        "financial": "Focus on financial metrics, revenue, profitability, and financial health.",
        "operations": "Focus on business operations, industry, products, and market position.",
        "general": "Provide a comprehensive overview of the company."
    }
    
    instruction = focus_instructions.get(focus, focus_instructions["general"])
    
    prompt = f"""You are a business analyst. Summarize the following company information. {instruction}

{company_context}

Summary:"""
    return prompt


def create_comparison_prompt(company1_context: str, company2_context: str, aspect: str = "general") -> str:
    """
    Create a comparison prompt for LLM.
    
    Parameters:
    -----------
    company1_context : str
        Formatted information for first company
    company2_context : str
        Formatted information for second company
    aspect : str
        Aspect to compare (e.g., "financial", "size", "industry")
        
    Returns:
    --------
    str
        Complete prompt for comparison task
    """
    prompt = f"""You are a comparative business analyst. Compare the following two companies, focusing on {aspect} aspects.

COMPANY 1:
{company1_context}

COMPANY 2:
{company2_context}

Comparison (focusing on {aspect}):"""
    return prompt


def create_extraction_prompt(company_context: str, fields_to_extract: list) -> str:
    """
    Create an information extraction prompt for LLM.
    
    Parameters:
    -----------
    company_context : str
        Formatted company information context
    fields_to_extract : list
        List of specific fields to extract
        
    Returns:
    --------
    str
        Complete prompt for extraction task
    """
    fields_str = ", ".join(fields_to_extract)
    prompt = f"""You are a data extraction specialist. Extract the following information from the company data and return it as JSON format.

Fields to extract: {fields_str}

{company_context}

Extracted information (JSON format):"""
    return prompt


# Example usage
if not df.empty:
    sample_context = row_to_context(df.iloc[0])
    
    print("="*80)
    print("PROMPT TEMPLATE EXAMPLES")
    print("="*80)
    
    print("\n1. QA Prompt:")
    print(create_qa_prompt(sample_context, "What industry does this company operate in?"))
    
    print("\n\n2. Classification Prompt:")
    print(create_classification_prompt(sample_context, ["Large Cap", "Mid Cap", "Small Cap", "Unknown"]))
    
    print("\n\n3. Summarization Prompt:")
    print(create_summarization_prompt(sample_context, "financial"))
    
    print("\n\n4. Extraction Prompt:")
    print(create_extraction_prompt(sample_context, ["revenue", "employees", "founded_year"]))


PROMPT TEMPLATE EXAMPLES

1. QA Prompt:
You are a financial analyst assistant. Answer the following question based on the company information provided.

COMPANY INFORMATION: 3M Company
--
Name: 3M Company
Type: Public
Industry: Conglomerate
Founded: in Two Harbors, Minnesota, U.S.
Headquarters: Maplewood, Minnesota
Employees: (2024)
Revenue: (2024)

Question: What industry does this company operate in?

Answer:


2. Classification Prompt:
You are a business classification expert. Classify the following company into one of these categories: Large Cap, Mid Cap, Small Cap, Unknown

COMPANY INFORMATION: 3M Company
--
Name: 3M Company
Type: Public
Industry: Conglomerate
Founded: in Two Harbors, Minnesota, U.S.
Headquarters: Maplewood, Minnesota
Employees: (2024)
Revenue: (2024)

Classification (choose one from: Large Cap, Mid Cap, Small Cap, Unknown):


3. Summarization Prompt:
You are a business analyst. Summarize the following company information. Focus on financial metrics, revenue, prof

Create a PromptBuilder class with different methods for creating prompts:
- qa prompt
- classification prompt
- summarization prompt
- comparison prompt
- information extraction prompt

In [25]:
# ============================================================================
# PROMPT BUILDER CLASS
# ============================================================================
# A class-based approach for building prompts with consistent structure

class PromptBuilder:
    """
    A class for building structured prompts for LLM tasks.
    Provides methods for different types of prompts with consistent formatting.
    """
    
    def __init__(self, system_role: str = "You are a helpful financial analyst assistant."):
        """
        Initialize the PromptBuilder.
        
        Parameters:
        -----------
        system_role : str
            System role/identity for the LLM
        """
        self.system_role = system_role
    
    def qa_prompt(self, company_context: str, question: str, include_examples: bool = False) -> str:
        """
        Create a question-answering prompt.
        
        Parameters:
        -----------
        company_context : str
            Formatted company information
        question : str
            Question to answer
        include_examples : bool
            Whether to include few-shot examples
            
        Returns:
        --------
        str
            Complete QA prompt
        """
        prompt = f"""{self.system_role}

Based on the following company information, answer the question.

{company_context}

Question: {question}

Answer:"""
        
        if include_examples:
            prompt = self._add_qa_examples(prompt)
        
        return prompt
    
    def classification_prompt(self, company_context: str, categories: list, 
                            task_description: str = "classify") -> str:
        """
        Create a classification prompt.
        
        Parameters:
        -----------
        company_context : str
            Formatted company information
        categories : list
            List of categories to choose from
        task_description : str
            Description of the classification task
            
        Returns:
        --------
        str
            Complete classification prompt
        """
        categories_str = ", ".join([f'"{cat}"' for cat in categories])
        prompt = f"""{self.system_role}

{task_description.capitalize()} the following company into one of these categories: {categories_str}

{company_context}

Classification (choose one from: {categories_str}):"""
        return prompt
    
    def summarization_prompt(self, company_context: str, focus: str = "general", 
                           length: str = "brief") -> str:
        """
        Create a summarization prompt.
        
        Parameters:
        -----------
        company_context : str
            Formatted company information
        focus : str
            Focus area (financial, operations, general)
        length : str
            Desired length (brief, detailed, comprehensive)
            
        Returns:
        --------
        str
            Complete summarization prompt
        """
        focus_map = {
            "financial": "Focus on financial metrics, revenue, profitability, and financial health.",
            "operations": "Focus on business operations, industry, products, and market position.",
            "general": "Provide a comprehensive overview."
        }
        
        length_map = {
            "brief": "Provide a brief summary (2-3 sentences).",
            "detailed": "Provide a detailed summary (1-2 paragraphs).",
            "comprehensive": "Provide a comprehensive summary covering all key aspects."
        }
        
        focus_instruction = focus_map.get(focus, focus_map["general"])
        length_instruction = length_map.get(length, length_map["brief"])
        
        prompt = f"""{self.system_role}

Summarize the following company information. {focus_instruction} {length_instruction}

{company_context}

Summary:"""
        return prompt
    
    def comparison_prompt(self, company1_context: str, company2_context: str, 
                         aspect: str = "general", output_format: str = "text") -> str:
        """
        Create a comparison prompt.
        
        Parameters:
        -----------
        company1_context : str
            First company's information
        company2_context : str
            Second company's information
        aspect : str
            Aspect to compare (financial, size, industry, etc.)
        output_format : str
            Desired output format (text, json, table)
            
        Returns:
        --------
        str
            Complete comparison prompt
        """
        format_instruction = {
            "text": "Provide a written comparison.",
            "json": "Provide the comparison in JSON format with fields: similarities, differences, conclusion.",
            "table": "Provide the comparison in a table format."
        }.get(output_format, "Provide a written comparison.")
        
        prompt = f"""{self.system_role}

Compare the following two companies, focusing on {aspect} aspects. {format_instruction}

COMPANY 1:
{company1_context}

COMPANY 2:
{company2_context}

Comparison:"""
        return prompt
    
    def information_extraction_prompt(self, company_context: str, fields_to_extract: list, 
                                    output_format: str = "json") -> str:
        """
        Create an information extraction prompt.
        
        Parameters:
        -----------
        company_context : str
            Formatted company information
        fields_to_extract : list
            List of fields to extract
        output_format : str
            Desired output format (json, csv, table)
            
        Returns:
        --------
        str
            Complete extraction prompt
        """
        fields_str = ", ".join(fields_to_extract)
        
        format_instruction = {
            "json": "Return the extracted information as a JSON object.",
            "csv": "Return the extracted information as CSV format.",
            "table": "Return the extracted information as a table."
        }.get(output_format, "Return the extracted information as a JSON object.")
        
        prompt = f"""{self.system_role}

Extract the following information from the company data: {fields_str}

{format_instruction}

{company_context}

Extracted Information:"""
        return prompt
    
    def _add_qa_examples(self, prompt: str) -> str:
        """Add few-shot examples to QA prompt."""
        examples = """

Examples:
Q: What is the company's revenue?
A: The company's revenue is $24.58 billion (2024).

Q: When was the company founded?
A: The company was founded in 1902."""
        return prompt.replace("Answer:", "Answer:" + examples)
    
    def batch_prompt(self, company_contexts: list, task: str, **kwargs) -> list:
        """
        Create multiple prompts for batch processing.
        
        Parameters:
        -----------
        company_contexts : list
            List of formatted company contexts
        task : str
            Task type (qa, classification, summarization, etc.)
        **kwargs
            Additional arguments for the specific task method
            
        Returns:
        --------
        list
            List of prompts
        """
        prompts = []
        method_map = {
            "qa": self.qa_prompt,
            "classification": self.classification_prompt,
            "summarization": self.summarization_prompt,
            "extraction": self.information_extraction_prompt
        }
        
        method = method_map.get(task.lower())
        if not method:
            raise ValueError(f"Unknown task: {task}. Available: {list(method_map.keys())}")
        
        for context in company_contexts:
            prompt = method(context, **kwargs)
            prompts.append(prompt)
        
        return prompts


# Example usage of PromptBuilder
if not df.empty:
    builder = PromptBuilder(system_role="You are an expert financial analyst with deep knowledge of corporate structures and markets.")
    
    sample_context = row_to_context(df.iloc[0])
    
    print("="*80)
    print("PROMPT BUILDER CLASS EXAMPLES")
    print("="*80)
    
    print("\n1. QA Prompt:")
    print(builder.qa_prompt(sample_context, "What is the company's primary industry?"))
    
    print("\n\n2. Classification Prompt:")
    print(builder.classification_prompt(
        sample_context, 
        ["Technology", "Finance", "Manufacturing", "Retail", "Other"],
        task_description="Classify the company's primary sector"
    ))
    
    print("\n\n3. Summarization Prompt (Financial Focus):")
    print(builder.summarization_prompt(sample_context, focus="financial", length="detailed"))
    
    print("\n\n4. Information Extraction Prompt:")
    print(builder.information_extraction_prompt(
        sample_context,
        ["revenue", "employees", "founded_year", "headquarters"],
        output_format="json"
    ))
    
    if len(df) >= 2:
        print("\n\n5. Comparison Prompt:")
        context2 = row_to_context(df.iloc[1])
        print(builder.comparison_prompt(sample_context, context2, aspect="financial", output_format="text"))


PROMPT BUILDER CLASS EXAMPLES

1. QA Prompt:
You are an expert financial analyst with deep knowledge of corporate structures and markets.

Based on the following company information, answer the question.

COMPANY INFORMATION: 3M Company
--
Name: 3M Company
Type: Public
Industry: Conglomerate
Founded: in Two Harbors, Minnesota, U.S.
Headquarters: Maplewood, Minnesota
Employees: (2024)
Revenue: (2024)

Question: What is the company's primary industry?

Answer:


2. Classification Prompt:
You are an expert financial analyst with deep knowledge of corporate structures and markets.

Classify the company's primary sector the following company into one of these categories: "Technology", "Finance", "Manufacturing", "Retail", "Other"

COMPANY INFORMATION: 3M Company
--
Name: 3M Company
Type: Public
Industry: Conglomerate
Founded: in Two Harbors, Minnesota, U.S.
Headquarters: Maplewood, Minnesota
Employees: (2024)
Revenue: (2024)

Classification (choose one from: "Technology", "Finance", "Manufa

In [ ]:
# ============================================================================
# STEP 7: SUMMARY & BEST PRACTICES GUIDE
# ============================================================================
# Comprehensive guide to using Wikipedia data with LLMs



"""
╔════════════════════════════════════════════════════════════════════════════╗
║                    LLM PROMPT ENGINEERING SUMMARY                         ║
╚════════════════════════════════════════════════════════════════════════════╝


🎯 BEST PRACTICES FOR LLM PROMPT INJECTION:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. CONTEXT QUALITY
   • Keep context focused and relevant
   • Clean and normalize text thoroughly
   • Remove ambiguous or conflicting information
   • Include metadata (source, confidence, date)

2. PROMPT DESIGN
   • Use clear, specific instructions
   • Provide examples when possible (few-shot)
   • Specify output format explicitly (JSON, tables, etc.)
   • Include role/perspective for better results



📋 USE CASES FOR YOUR DATA:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✓ Company Classification          → Categorize by industry, size, sector
✓ Market Analysis                → Competitive landscape, positioning
✓ Risk Assessment               → Financial health, strategic risks
✓ Investment Analysis           → Potential returns, growth prospects
✓ Data Enrichment               → Fill gaps from Wikipedia data
✓ Text Generation               → Create summaries, reports, profiles
✓ Knowledge Extraction          → Key metrics, relationships, entities
✓ Sentiment Analysis            → Company reputation, public perception
✓ Trend Detection               → Emerging patterns, growth areas
✓ Comparative Analysis          → Company benchmarking, peer analysis
"""


End of lab 5

🚀 NEXT STEPS (in anticipation of the final lab)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. Export your full dataset using PromptExporter
2. Test prompts with a small sample (5-10 companies)
3. Evaluate LLM outputs for quality and accuracy
4. Iterate on prompts based on results
5. Scale up to full dataset using batch APIs
6. Monitor token usage and costs
7. Implement feedback loops for continuous improvement
8. Build evaluation metrics for output quality